Reset Camera

In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon 

[sudo] password for jetson: 

Change Directory

In [2]:
%cd /home/jetson/fast-and-furious-with-self-drive-ai/notebooks/road_following/

/home/jetson/fast-and-furious-with-self-drive-ai/notebooks/road_following


# Road Following Live (with camera)

Load the optimized model (created with the [`optimize_model.ipynb` notebook](./optimize_model.ipynb)) executing the cell below

In [ ]:
import torch
from torch2trt import TRTModule
import numpy as np
from collections import deque

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trained_models/updated_model_trt.pth'))

prev_x = deque(maxlen=5)

Create the racecar class

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

In [ ]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

In [ ]:
def road_confidence(cv_image):
    """Calculate confidence based on road color detection"""
    colour_img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([40, 60, 60])
    upper_green = np.array([80, 255, 255])
    mask = cv2.inRange(colour_img, lower_green, upper_green)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    height, width = mask.shape
    region = mask[int(height * 0.6):, int(width * 0.3):int(width * 0.7)]
    green_pixels = cv2.countNonZero(region)
    area = region.shape[0] * region.shape[1]
    return green_pixels / area if area > 0 else 0.0

def ml_confidence():
    """Calculate confidence based on steering stability"""
    if len(prev_x) < 2:  # Need at least 2 values for std
        return 1.0
    return 1.0 - np.var(list(prev_x)[-5:])

Before letting your car go, let's prepare slide bars for gains and offset, so that you can adjust them during the runtime.

In [ ]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

road_conf_output_slider = widgets.FloatSlider(description='Road Confidence', min=0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
ml_conf_output_slider = widgets.FloatSlider(description='ML Confidence', min=0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=1.0, value=-1.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
#throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.0, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([road_conf_output_slider,
                       widgets.Label(value="X"),
                       ml_conf_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         ]
        )
)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    road_conf = road_confidence(new_image)

    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    prev_x.append(x)

    ml_conf = ml_confidence()
    
    ml_conf_output_slider.value = ml_conf
    road_conf_output_slider.value = road_conf
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    if(steering<-1.0):
        steering_value_slider.value = -1.0
    elif(steering>1.0):
        steering_value_slider.value = 1.0
    else:
        steering_value_slider.value = steering 
    car.steering = x
    
    k = 2
    car.throttle = 0.55*np.exp(-abs(x) * k)
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
update({'new': camera.value})  # we call the function once to initialize

Run Live Camera Feed

In [ ]:
camera.observe(update, names='value') 

In [ ]:
camera.running = True

Start Motors

In [ ]:
car.throttle_gain = 1

Stop Motors

In [ ]:
car.throttle_gain = 0